# problem 1

In [ ]:
import numpy as np
import scipy.stats as st
from matplotlib import pyplot

In [ ]:
def mandelbrot(iters, samples, x_min = -2, x_max = 1, y_min = -1.5, y_max = 1.5):
    
    max_iter = iters
    
    width, height = 800, 800
    mandelbrot_set = np.zeros((height, width))

    inside_count = 0

    for i in range(int(samples)):
        real = np.random.uniform(x_min, x_max)
        imag = np.random.uniform(y_min, y_max)
        c = complex(real, imag)
        z = 0.0j
        for j in range(max_iter):
            z = z*z + c
            if (z.real*z.real + z.imag*z.imag) >= 4:
                break
        if j == max_iter - 1:
            inside_count += 1
            x_pixel = int((real - x_min) * width / (x_max - x_min))
            y_pixel = int((imag - y_min) * height / (y_max - y_min))
            mandelbrot_set[y_pixel, x_pixel] = 1
    area_estimate = (inside_count / samples) * (x_max - x_min) * (y_max - y_min)

    return mandelbrot_set, area_estimate

In [ ]:
width, height = 800, 800
max_iter = 100

x_min, x_max = -2, 1
y_min, y_max = -1.5, 1.5

mandelbrot_set = np.zeros((height, width))

samples = 1000000

inside_count = 0

mandelbrot_set, mandelbrot_area = mandelbrot(max_iter, samples)

print(mandelbrot_area)

pyplot.imshow(mandelbrot_set, extent=(x_min, x_max, y_min, y_max), cmap="hot")
pyplot.colorbar()
pyplot.title("Mandelbrot Set")
pyplot.xlabel("Real")
pyplot.ylabel("Imaginary")
pyplot.savefig("fractal", dpi=300)

In [ ]:
fig, axes = pyplot.subplots(3, 3,sharex=True,sharey=True,figsize=(20,20))

iter_list = [40, 70, 100]
sample_list = [1e4, 1e5, 1e6]

x_min, x_max = -2, 1
y_min, y_max = -1.5, 1.5

for iters in range(len(iter_list)):
    for sample in range(len(sample_list)):
        mandelbrot_set, _ = mandelbrot(iter_list[iters], sample_list[sample])
        axes[iters,sample].imshow(mandelbrot_set, extent=(x_min, x_max, y_min, y_max), cmap="hot")
        axes[2,sample].set_xlabel(f"Real\nSample = {int(sample_list[sample])}", fontsize=14)
        axes[iters,0].set_ylabel(f"Iterations = {iter_list[iters]}\nImaginary", fontsize=14)

fig.suptitle("Mandelbrot Set Changing Iterations and Samples", fontsize=25)
pyplot.tight_layout()
pyplot.savefig("sub_fractal", dpi=300)

# Problem 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# step 1
# Define a function that determines whether a point is within the Mandelbrot set
def is_in_mandelbrot(c, max_iter):
    """
    Check if the complex number c is in the Mandelbrot set.
    
    Parameters:
    c (complex): The complex number to check.
    max_iter (int): The maximum number of iterations to perform.
    
    Returns:
    bool: True if c is in the Mandelbrot set, False otherwise.
    """
    z = 0
    for n in range(max_iter):
        z = z*z + c
        if abs(z) > 2:
            return False, n
    return True, max_iter

# The Function of Estimating Area of Monte Carlo Method
def monte_carlo_mandelbrot_area(xmin, xmax, ymin, ymax, num_samples, max_iter):
    count_inside = 0
    for _ in range(num_samples):
        real = np.random.uniform(xmin, xmax)
        imag = np.random.uniform(ymin, ymax)
        c = complex(real, imag)
        in_mandelbrot, _ = is_in_mandelbrot(c, max_iter)
        if in_mandelbrot:
            count_inside += 1
    rect_area = (xmax - xmin) * (ymax - ymin)
    mandelbrot_area_estimate = (count_inside / num_samples) * rect_area
    return mandelbrot_area_estimate

# Set up parameters
xmin, xmax, ymin, ymax = -2.0, 1.0, -1.5, 1.5
num_samples = 10000
max_iter = 100

# Estimated area of Mandelbrot collection
estimated_area = monte_carlo_mandelbrot_area(xmin, xmax, ymin, ymax, num_samples, max_iter)
estimated_area

In [ ]:
# Step 2
# Analysis of Area Estimation Convergence under Different Iterations i and Sample Numbers s

# Define the number of iterations and the number of samples
iter_list = [100, 500, 1000, 5000]  # Iteration list
sample_list = [1000, 5000, 10000, 50000]  # Sample numbers lists

# Build a dictionary
area_estimates = {}

# Each combination of number of iterations and number of samples is estimated by Monte Carlo
for max_iter in iter_list:
    for num_samples in sample_list:
        estimate = monte_carlo_mandelbrot_area(xmin, xmax, ymin, ymax, num_samples, max_iter)
        area_estimates[(max_iter, num_samples)] = estimate

area_estimates

# prepare data to plot
iter_sample_area = np.array([(key[0], key[1], value) for key, value in area_estimates.items()])

# Area estimates are plotted grouped by the number of iterations and the number of samples
fig, ax = plt.subplots(figsize=(14, 8))

for max_iter in iter_list:
    # Extract the sample number and area estimates for the current iteration number from iter_sample_area
    data = iter_sample_area[iter_sample_area[:, 0] == max_iter]
    ax.plot(data[:, 1], data[:, 2], '-o', label=f'max_iter = {max_iter}')

ax.set_xlabel('Number of samples (s)', fontsize=14)
ax.set_ylabel('Estimated Area of Mandelbrot Set', fontsize=14)
ax.set_title('Convergence of Monte Carlo Estimation of Mandelbrot Set Area', fontsize=16)
ax.legend()
ax.grid(True)

plt.savefig("1",dpi=300)

In [ ]:
# Step3
# Select a large number of iterations i as a reference
i_reference = 5000

# Select the number of samples s
s_samples = 10000

# Calculate the area estimation Ai,s under the number of reference iterations i
A_i_s_reference = monte_carlo_mandelbrot_area(xmin, xmax, ymin, ymax, s_samples, i_reference)

# Generates a list with a smaller number of iterations j, less than i
j_list = [100, 500, 1000, 2000, 3000, 4000]

# Calculate the area under each j and estimate Aj,s
A_j_s_values = [monte_carlo_mandelbrot_area(xmin, xmax, ymin, ymax, s_samples, j) for j in j_list]

# Calculation variance (Aj, S-AI,s)
differences = [A_j_s - A_i_s_reference for A_j_s in A_j_s_values]

# Output result
A_i_s_reference, A_j_s_values, differences

# Polt
plt.figure(figsize=(10, 6))

# Plot the difference as j increases
plt.plot(j_list, differences, '-o', label='Difference (Aj,s - Ai,s)')

plt.axhline(y=0, color='black', linewidth=1)

# Set chart titles and axis labels
plt.title('Convergence of Area Estimates as a Function of Iteration Count')
plt.xlabel('Iteration Count (j)')
plt.ylabel('Difference from Reference Area Estimate (Aj,s - Ai,s)')
plt.legend()
plt.grid(True)

# Show chart
plt.savefig("2",dpi=30)

In [ ]:
# Step4
# Repeat tests
# Number of repetitions for the experiment
num_repetitions = 50

# Initialize arrays to store the cumulative differences for each j value
cumulative_differences = np.zeros(len(j_list))

# Repeat the experiment num_repetitions times
for _ in range(num_repetitions):
    A_i_s_reference = monte_carlo_mandelbrot_area(xmin, xmax, ymin, ymax, s_samples, i_reference)
    A_j_s_values = [monte_carlo_mandelbrot_area(xmin, xmax, ymin, ymax, s_samples, j) for j in j_list]
    differences = np.array(A_j_s_values) - A_i_s_reference
    cumulative_differences += differences

# Calculate average differences
average_differences = cumulative_differences / num_repetitions

# Plotting the average differences
plt.figure(figsize=(10, 6))
plt.bar([str(j) for j in j_list], average_differences, color='skyblue')
plt.xlabel('Number of Iterations (j)')
plt.ylabel('Average Difference from Reference')
plt.title('Average Differences in Mandelbrot Area Estimation for Varying Iterations')
plt.savefig("3",dpi=30)

# problem 3

In [ ]:
import time
import numpy as np
import scipy.stats as st
from matplotlib import pyplot

In [ ]:
def pure_random_sampling(samples, x_min = -2, x_max = 1, y_min = -1.5, y_max = 1.5):
    
    real_samples = np.random.uniform(x_min, x_max, samples)
    imag_samples = np.random.uniform(y_min, y_max, samples)
    
    return real_samples, imag_samples


def latin_hypercube_sampling(samples, dimensions, x_min = -2, x_max = 1, y_min = -1.5, y_max = 1.5):
    
    cut_points = np.linspace(0, 1, samples + 1)
    margins = cut_points[:-1]

    real_samples = margins + np.random.rand(samples) * (1/samples)
    imag_samples = margins + np.random.rand(samples) * (1/samples)

    np.random.shuffle(imag_samples)

    real_samples = real_samples * (x_max - x_min) + x_min
    imag_samples = imag_samples * (y_max - y_min) + y_min
    
    return real_samples, imag_samples


def orthogonal_sampling(samples_per_dimension, x_min = -2, x_max = 1, y_min = -1.5, y_max = 1.5):

    sample_points = np.arange(0, samples_per_dimension)
    
    real_samples = np.zeros(samples_per_dimension**2)
    imag_samples = np.zeros(samples_per_dimension**2)
    
    for i in range(samples_per_dimension):
        np.random.shuffle(sample_points)
        for j in range(samples_per_dimension):
            idx = i * samples_per_dimension + j
            real_samples[idx] = sample_points[j] / samples_per_dimension + np.random.rand() / samples_per_dimension
            imag_samples[idx] = i / samples_per_dimension + np.random.rand() / samples_per_dimension
    
    np.random.shuffle(real_samples)
    np.random.shuffle(imag_samples)
    
    real_samples = real_samples * (x_max - x_min) + x_min
    imag_samples = imag_samples * (y_max - y_min) + y_min
    
    return real_samples, imag_samples

def mandelbrot_diff_sampling(iters, samples, real_samples, imag_samples, x_min = -2, x_max = 1, y_min = -1.5, y_max = 1.5):
    max_iter = iters
    width, height = 800, 800
    mandelbrot_set = np.zeros((height, width))

    cs = real_samples + 1j * imag_samples
    zs = np.zeros(samples, dtype=complex)

    inside = np.ones(samples, dtype=bool)

    for i in range(max_iter):
        zs = zs * zs + cs
        escape = np.abs(zs) >= 2.0
        zs[escape] = 0
        inside &= ~escape

    inside_count = np.sum(inside)

    x_pixels = np.clip(((real_samples - x_min) * width / (x_max - x_min)).astype(int), 0, width - 1)
    y_pixels = np.clip(((real_samples - y_min) * height / (y_max - y_min)).astype(int), 0, height - 1)
    mandelbrot_set[y_pixels, x_pixels] += inside

    area_estimate = (inside_count / samples) * (x_max - x_min) * (y_max - y_min)
    
    return area_estimate

In [ ]:
samples = int(1e6)
dimensions = 2
iterations = 80
repeat_times = 100

mean_values = []
std_deviation_values = []
confidence_interval_values = []
sampling_methods = []

for sampling_method, function, params in [
    ("pure random sampling", pure_random_sampling, dict(samples=samples)),
    ("latin hypercube sampling", latin_hypercube_sampling, dict(samples=samples, dimensions=dimensions)),
    ("orthogonal sampling", orthogonal_sampling, dict(samples_per_dimension=int(samples**(1/2))))
]:
    area_estimates = []
    for _ in range(repeat_times):
        real_samples, imag_samples = function(**params)   
        area_estimate = mandelbrot_diff_sampling(iters=iterations, samples=samples,
                                               real_samples=real_samples, imag_samples=imag_samples)
        area_estimates.append(area_estimate)
        
    mean = np.mean(area_estimates)
    std_deviation = np.std(area_estimates, ddof=1)
    
    mean_values.append(mean)
    std_deviation_values.append(std_deviation)
    sampling_methods.append(sampling_method)
    
    confidence_level = 0.95
    degrees_freedom = len(area_estimates) - 1
    confidence_interval = st.norm.interval(confidence_level, loc=mean, scale=std_deviation)
    confidence_interval_values.append(confidence_interval)

    print(f"For {sampling_method}, the mean area estimate is {mean}.")
    print(f"The standard deviation is {std_deviation}.")
    print(f"{confidence_level*100}% confidence interval is {confidence_interval}.")
    
    
fig, ax = pyplot.subplots(figsize=(10, 6))

bar_positions = range(len(mean_values))
ax.bar(bar_positions, mean_values, yerr=std_deviation_values, color="lightblue", align="center", alpha=0.7, lw=2, ecolor="grey", capsize=8)
    
for i, conf_interval in enumerate(confidence_interval_values):
    ax.plot([bar_positions[i], bar_positions[i]], conf_interval, color="royalblue", marker='_', markersize=20)
    
ax.set_ylim([1.545, 1.570])
ax.set_xticks(bar_positions, sampling_methods)
ax.set_ylabel("Mandelbrot Set Area Estimate", fontsize=13)
ax.set_xlabel("Sampling Method", fontsize=13)
ax.set_title("Comparison of Mandelbrot Set Area Estimates", fontsize=18)
pyplot.tight_layout()
pyplot.savefig("sampling", dpi=300)


samples = int(1e6)
dimensions = 2
iterations = 80
repeat_times = 100
alpha = 0.001

mean_values = []
mean_values_cvg = []
sampling_methods = []

for sampling_method, function, params in [
    ("pure random sampling", pure_random_sampling, dict(samples=samples)),
    ("latin hypercube sampling", latin_hypercube_sampling, dict(samples=samples, dimensions=dimensions)),
    ("orthogonal sampling", orthogonal_sampling, dict(samples_per_dimension=int(samples**(1/2))))
]:
    area_estimates = []
    area_estimates_cvg = []
    start_time = time.time()
    for _ in range(repeat_times):
        real_samples, imag_samples = function(**params)
        area_estimate = mandelbrot_diff_sampling(iters=iterations, samples=samples,
                                               real_samples=real_samples, imag_samples=imag_samples)
       
        area_estimates.append(area_estimate)
        mean = np.mean(area_estimates)
        mean_values.append(mean)
        sampling_methods.append(sampling_method)
        
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    for _ in range(repeat_times):
        real_samples, imag_samples = function(**params)
        area_estimate = mandelbrot_diff_sampling(iters=iterations, samples=samples,
                                               real_samples=real_samples, imag_samples=imag_samples)
       
        area_estimates_cvg.append(area_estimate)
        mean = np.mean(area_estimates_cvg)
        mean_values_cvg.append(mean)
        
    mean_H0 = np.mean(mean_values_cvg)
    t_statistic, p_value = st.ttest_1samp(mean_values, mean_H0)
    
    print(f"For {sampling_method}, t-statistic: {t_statistic}, p-value: {p_value}, Computation uses {elapsed_time}s.")

    if p_value < alpha:
        print(f"Reject the null hypothesis at a significance level of {alpha}.")
    else:
        print(f"Accept the null hypothesis at a significance level of {alpha}.")

# problem 4

In [ ]:
def mandelbrot_pre(iters, samples, x_min=-2, x_max=1, y_min=-1.5, y_max=1.5):
    max_iter = iters
    inside_count = 0

    start_time = time.time()

    for _ in range(int(samples)):
        real = np.random.uniform(x_min, x_max)
        imag = np.random.uniform(y_min, y_max)
        c = complex(real, imag)
        z = 0.0j

        for j in range(max_iter):
            z = z * z + c
            if (z.real * z.real + z.imag * z.imag) >= 4:
                break
        if j == max_iter - 1:
            inside_count += 1

    end_time = time.time()

    elapsed_time = end_time - start_time

    return elapsed_time


def mandelbrot_optimized(iters, samples, x_min=-2, x_max=1, y_min=-1.5, y_max=1.5):
    max_iter = iters
    inside_count = 0

    start_time = time.time()

    for _ in range(samples):
        real = np.random.uniform(x_min, x_max)
        imag = np.random.uniform(y_min, y_max)
        x, y = real, imag
        x2, y2 = x * x, y * y
        iteration = 0

        while x2 + y2 <= 4 and iteration < max_iter:
            y = 2 * x * y + imag
            x = x2 - y2 + real
            x2, y2 = x * x, y * y
            iteration += 1

        if iteration == max_iter:
            inside_count += 1

    end_time = time.time()

    elapsed_time = end_time - start_time

    return elapsed_time

In [ ]:
iteration_list = np.linspace(100,300,10)
time_diff_list = []
samples = int(1e5)

for i in iteration_list:
    i = int(i)
    time_diff = mandelbrot_pre(i,samples) - mandelbrot_optimized(i,samples)
    time_diff_list.append(time_diff)
    
pyplot.plot(iteration_list, time_diff_list)
pyplot.axhline(y=0, color="royalblue", linestyle='--', linewidth=2)
pyplot.xlabel("Iterations")
pyplot.ylabel("Time/s")
pyplot.title("Time Shortened Using Importance Sampling Method")
pyplot.savefig("time_diff_fix_s", dpi=300)

In [ ]:
sample_list = np.linspace(1e5,1e6,10)
time_diff_list = []
iterations = 100

for s in sample_list:
    s = int(s)
    time_diff = mandelbrot_pre(iterations,s) - mandelbrot_optimized(iterations,s)
    time_diff_list.append(time_diff)
    
pyplot.plot(sample_list, time_diff_list)
pyplot.axhline(y=0, color="royalblue", linestyle='--', linewidth=2)
pyplot.xlabel("Samples")
pyplot.ylabel("Time/s")
pyplot.title("Time Shortened Using Importance Sampling Method")
pyplot.savefig("time_diff_fix_i", dpi=300)